<h1><center> SEN163A - Fundamentals of Data Analytics </center>
<center> Assignment 2 - Large Scale Internet Data Analysis  </center></h1>

<center> Lecture:
Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl) </center>

## Group 3
- Hilmy Hanif - 5270243
- Terence Celestina - 1524658
- Leopoldo Ochoa Diaz - 5256992
- Ivy de Jong - 4932935

## Outline:
1. [Introduction](#1.-Introduction)
2. [Data Description](#2.-Data Description)
3. [Formulation](#3.-Formulation)
4. [Conclusion](#4.-Conclusion)
5. [Reference](#5.-Reference)

## 1. Introduction

### Definition

An autonomous system number is a unique identifier that is globally available and allows its autonomous system to exchange routing information with other systems.

### Overview

The Groote Nationale Investeer Bank (GNI Bank) is a large European bank and is preparing to enter the
mobile banking sector. They have hired your advisory firm to prepare a strategy. Your data analytics team
has been put on the task of finding the best four datacenter locations for hosting in the EU.


### Requested tasks:
- Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.
- With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last three in your report (number, name and country).
- For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and the target IPv4 is from an EU country. Implement this in an efficient way.
- Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then, using all processed files, calculate the average latency’s for each country-AS combination and store the results into one ncountries ×m matrix. If we could place one server in each country, what would the minimum average latency be for each country? (include in your report)
- Since we are only allowed to place four servers, determine the best four datacenters based on the total latency for all countries. Report your findings and your procedure to obtain them. Also include the average latency for each country.


## 2. Data Description

### Task A
Evaluate AS data and Probe Data

## AS Data

In [1]:
import pickle
import pandas as pd
import ipaddress
import numpy as np

# Open the file in binary mode
with open('AS_dataset.pkl', 'rb') as file: #rb means in read mode
	
	# Call load method to deserialze
	AS_df = pickle.load(file)

In [2]:
AS_df.shape #know columns and rows of the AS dataframe

(60122, 5)

In [3]:
AS_df.head(10) #show first 10 row of the dataset

ASN Country                                            Name  NumIPs  \
0   AS55330      AF  AFGHANTELECOM GOVERNMENT COMMUNICATION NETWORK  50,432   
1   AS17411      AF                 Io Global Services Pvt. Limited  13,568   
2   AS55424      AF                            Instatelecom Limited  13,312   
3   AS38742      AF                                            AWCC  11,520   
4  AS131284      AF                                 Etisalat Afghan  10,240   
5   AS45178      AF                                       ROSHAN-AF   5,376   
6  AS132471      AF                                  MTNAFGHANISTAN   4,864   
7    AS7494      AF                                CeReTechs Co ltd   4,096   
8  AS138322      AF                                 Afghan Wireless   3,584   
9   AS55745      AF                         Neda Telecommunications   2,560   

       type  
0   hosting  
1  business  
2  business  
3       isp  
4       isp  
5  business  
6  business  
7  business  
8  business  
9  business

In [4]:
AS_df.tail(10) #show last 10 row of the dataset

ASN Country                        Name NumIPs       type
60112  AS327863      ZW      University of Zimbabwe    256  education
60113  AS328235      ZW  Zimbabwe Internet Exchange    256        isp
60114   AS37329      ZW         Frampol Investments      0        isp
60115   AS27576      ZW      AFRICA ONLINE ZIMBABWE      0        isp
60116  AS328291      ZW  Veritran (Private) Limited      0        isp
60117   AS37332      ZW             Zimbabwe Online      0  education
60118   AS37344      ZW    ZIMBABWE OPEN UNIVERSITY      0   business
60119   AS37355      ZW  Zimbabwe Internet Exchange      0   business
60120   AS37391      ZW          Esmalo Enterprises      0   business
60121   AS37485      ZW    Data Control and Systems      0   business

### AS Column Description

- ASN : autonomus system number
- Country : specific country that is represented by the ASN
- Name : name of the company that use the ASN
- NumIPs : number of IPs used for the ASN
- Type : type of purpose of the ASN

In [5]:
AS_df.describe() #summarize description of the AS dataframe

ASN Country     Name NumIPs      type
count     60122   60122    60122  60122     60122
unique    60122     237    51848   1673         5
top     AS55330      DE  Renater      0  business
freq          1    2500      307  15107     33612

From the table above, the NumIPs = 0 probably indicates that specified ASN is not used. In the next step, the NumIPs = 0 could be filtered from the data in order to use active ASN only.

In [6]:
AS_df[AS_df['NumIPs'] == '0'].count() / AS_df['NumIPs'].count() 
#identify NumIPs that is equal to 0, which is not utilized.

ASN        0.251272
Country    0.251272
Name       0.251272
NumIPs     0.251272
type       0.251272
dtype: float64

There are 25,12% of 0 NumIPs in the AS dataset. Take a note that later the NumIPs 0 should be filtered later to use only valid values.

In [7]:
AS_df['Country'].value_counts() #count total country in the dataset

DE    2500
BR    2500
PL    2500
IN    2500
RU    2500
      ... 
UM       1
AQ       1
FK       1
MP       1
KP       1
Name: Country, Length: 237, dtype: int64

There are 237 countries ranging from different continent. However this dataset only represents the country, not including the continent. To observe EU only, there should be another additional data point used to utilized only ASN from EU countries.

In [8]:
AS_df['type'].value_counts() #count value of type

business     33612
unk          11052
isp          10989
hosting       2847
education     1622
Name: type, dtype: int64

In [9]:
AS_df[AS_df['NumIPs'] != '0']['type'].value_counts() #identify hosting type with NumIPs is not 0.

business     26835
isp           8739
unk           5839
hosting       2310
education     1292
Name: type, dtype: int64

There are 5 types of the ASN purpose. Later, it is obligated to filter the dataset only for 'hosting' type which consists of 2310 data.

In [10]:
AS_df.info() 
#limitation: different data types between AS and Probe data should be converted into same data type. Data from pickle should be converted first
#format pkl data type provides all same object datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60122 entries, 0 to 60121
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ASN      60122 non-null  object
 1   Country  60122 non-null  object
 2   Name     60122 non-null  object
 3   NumIPs   60122 non-null  object
 4   type     60122 non-null  object
dtypes: object(5)
memory usage: 2.3+ MB


In [11]:
AS_df = AS_df.astype(dtype={'ASN': 'string'}) #convert the data type of ASN from object to string

In [12]:
AS_df['NumIPs'] = AS_df['NumIPs'].str.replace(',', '').astype(float).astype(int)
#remove comma between numbers
#convert to float
#convert to int

In [13]:
AS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60122 entries, 0 to 60121
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ASN      60122 non-null  string
 1   Country  60122 non-null  object
 2   Name     60122 non-null  object
 3   NumIPs   60122 non-null  int32 
 4   type     60122 non-null  object
dtypes: int32(1), object(3), string(1)
memory usage: 2.1+ MB


In [14]:
AS_df.isnull().any() #find if there are any null data

ASN        False
Country    False
Name       False
NumIPs     False
type       False
dtype: bool

There are no null data for this AS dataset

## Probe Data

In [15]:
# Open the file in binary mode
with open('probe_dataset.pkl', 'rb') as file:
	
	# Call load method to deserialze
	probe_df = pickle.load(file)

In [16]:
probe_df.shape #identify row and column of the Probe Data. There are 2 columns and 11008 rows.

(11008, 2)

In [17]:
probe_df.head(10) #show first 10 data of Probe Data

prb_id      ASN
0      1   AS3265
1      2   AS1136
2      3   AS3265
3      6   AS6830
4      8   AS3265
5     11  AS12333
6     14   AS3269
7     20   AS3265
8     24   AS7922
9     26   AS3265

### Probe Data Column Description

- prb_id : probe id. The probe id of the probe data is correspond to ASN. A probe is located in ASN which ASN contains information of the localization and the type.
- ASN : autonomus system number

In [18]:
probe_df = probe_df.astype(dtype={'ASN': 'string'})
probe_df = probe_df.astype(dtype={'prb_id': 'int'})
#convert the ASN and prb_id data type to string and integer

In [19]:
probe_df.info() #the datatype of prb_id and ASN is changed.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11008 entries, 0 to 11007
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   prb_id  11008 non-null  int32 
 1   ASN     11008 non-null  string
dtypes: int32(1), string(1)
memory usage: 129.1 KB


In [20]:
probe_df.describe() #describe the prb_id dataset

prb_id
count  1.100800e+04
mean   4.017255e+04
std    1.210076e+05
min    1.000000e+00
25%    1.233175e+04
50%    2.318700e+04
75%    3.387425e+04
max    1.000246e+06

Unpickling: It is the inverse of Pickling process where a byte stream is converted into an object hierarchy. Therefore, the IP numbers is in integer. (?)

## RIPE Data (for one hour)

In [21]:
import os
import bz2
import time
import json
import glob
pd.set_option('display.max_columns', None)

In [22]:
#temp = pd.DataFrame()

#path_to_json ='https://data-store.ripe.net/datasets/atlas-daily-dumps/2022-03-01/ping-2022-03-01T0000' 

#json_pattern = os.path.join(path_to_json,'*.bz2')
#file_list = glob.glob(json_pattern)

#for file in file_list:
#    chunks = pd.read_json(file, lines=True, chunksize=1000)
#    i = 0
#    chunk_list = []
#    for chunk in chunks:
#        if i >= 10:
#            break
#        i += 1
#        chunk_list.append(chunk)
#        df = pd.concat(chunk_list, sort = True)
#    temp = temp.append(df, sort = True)
#temp

In [23]:
bz2Filename = 'ping-2022-03-01T0000.bz2'
bz2File     = bz2.open(bz2Filename, 'rt') 

# List of tuples
# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
tuple_list = []
#read first 10 lines to estimate total loading time
totalNrOfLines = 0;
filteredNrOfLines = 0

st  = time.time()
for line in bz2File:
    #print(line)
    decoded_line = json.loads(line)
    if(decoded_line["af"] == 4):
        filteredNrOfLines += 1
        tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
    print(decoded_line)
    totalNrOfLines += 1
    if totalNrOfLines>10: break

#print time and estimate total time            
dur         = round(time.time() - st,2)
estTotTime  = round( (dur/100000)*totalNrOfLines )
print("\nbz2 file:" )
print("Loading 100k lines took: "  + str(dur) + " seconds")
print("Estimated loading time of entire bz2 file: "  + str(estTotTime) + \
      " seconds" )

#finally close bz2File
bz2File.close()

{'fw': 5040, 'mver': '2.4.1', 'lts': 6, 'dst_name': '156.154.79.254', 'af': 4, 'dst_addr': '156.154.79.254', 'src_addr': '185.144.238.12', 'proto': 'ICMP', 'ttl': 56, 'size': 64, 'result': [{'rtt': 27.987967}, {'rtt': 28.039935}, {'rtt': 28.146594}], 'dup': 0, 'rcvd': 3, 'sent': 3, 'min': 27.987967, 'max': 28.146594, 'avg': 28.0581653333, 'msm_id': 29692049, 'prb_id': 26262, 'timestamp': 1646093125, 'msm_name': 'Ping', 'from': '185.144.238.12', 'type': 'ping', 'group_id': 29692048, 'step': 240}
{'fw': 5040, 'mver': '2.4.1', 'lts': 10, 'dst_name': '2001:620:410:ffad::2', 'af': 6, 'dst_addr': '2001:620:410:ffad::2', 'src_addr': '2a07:4400:0:1001:eade:27ff:fec9:67a4', 'proto': 'ICMP', 'ttl': 54, 'size': 64, 'result': [{'rtt': 11.337037}, {'rtt': 11.336182}, {'rtt': 11.19329}], 'dup': 0, 'rcvd': 3, 'sent': 3, 'min': 11.19329, 'max': 11.337037, 'avg': 11.2888363333, 'msm_id': 19215818, 'prb_id': 26262, 'timestamp': 1646093129, 'msm_name': 'Ping', 'from': '2a07:4400:0:1001:eade:27ff:fec9:67a

The differences of dst_name and dst_address:
Dst_address: IP address of the destination (string)
Dst_name: name of the destination (string), could be the same as dst_address if it is not hosted yet, but the dst_name usually utilized as example: google.com, facebook.comw which actually represent the IP address.

In [24]:
#bz2Filename = 'ping-2022-03-01T0000.bz2'
#bz2File     = bz2.open(bz2Filename, 'rt')

#read first 10 lines to estimate total loading time
#count = 0;
#st    = time.time()
#for line in bz2File:
#   print(line)
#    decoded_line = json.loads(line)
#    print(decoded_line)
#    count = count + 1
#    if count>10: break

#print time and estimate total time            
#dur         = round(time.time() - st,2)
#estTotTime  = round( (dur/100000)*nrOfLines )
#print("\nbz2 file:" )
#print("Loading 100k lines took: "  + str(dur) + " seconds")
#print("Estimated loading time of entire bz2 file: "  + str(estTotTime) + \
#      " seconds" )

#finally close bz2File
#bz2File.close()

### IPv4 data

This site or product includes IP2Location LITE data available from <a href="https://lite.ip2location.com">https://lite.ip2location.com</a>.

List of EU countries (see reference):
https://european-union.europa.eu/principles-countries-history/country-profiles_en?page=1

In [25]:
ipv4_df = pd.read_csv("IP2LOCATION-LITE-DB1.CSV")

In [26]:
ipv4_df.shape #identify total rows and columns in IPv4 data

(212146, 4)

In [27]:
ipv4_df.head() #display first 5 data of ipv4 data

0  16777215   -                       -.1
0  16777216  16777471  US  United States of America
1  16777472  16778239  CN                     China
2  16778240  16779263  AU                 Australia
3  16779264  16781311  CN                     China
4  16781312  16785407  JP                     Japan

In [28]:
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True) #add column name

In [29]:
ipv4_df.head() #display data with column name

ip_from     ip_to country_code              country_name
0  16777216  16777471           US  United States of America
1  16777472  16778239           CN                     China
2  16778240  16779263           AU                 Australia
3  16779264  16781311           CN                     China
4  16781312  16785407           JP                     Japan

### IPv4 Data Column Description

- ip_from: lowest range of the IP in the country
- ip_to: highest range of the IP in the country
- country_code: code of country with range of IP's
- country_name: name of country with range of IP's

In [30]:
ipv4_df.info() #collect information of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212146 entries, 0 to 212145
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ip_from       212146 non-null  int64 
 1   ip_to         212146 non-null  int64 
 2   country_code  212089 non-null  object
 3   country_name  212146 non-null  object
dtypes: int64(2), object(2)
memory usage: 6.5+ MB


### Task B
Evaluate the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set.

In [31]:
#merge the AS dataset and Probe data
AS_pb_joined = probe_df.merge(AS_df, left_on='ASN', right_on='ASN', how='inner')
AS_pb_joined = AS_pb_joined.replace(np.nan,'', regex=True) 

In [32]:
AS_pb_joined.tail() #show last 5 of the merged dataset

prb_id      ASN Country  \
10700  1000216  AS41620      IR   
10701  1000222  AS60610      GB   
10702  1000226  AS61098      CH   
10703  1000237   AS7489      US   
10704  1000242  AS49432      AT   

                                                  Name  NumIPs       type  
10700        Iran University Of Science and Technology    4096   business  
10701                                        Amito Ltd    9984    hosting  
10702                                        Akenes SA   22528   business  
10703                                           HostUS   59392  education  
10704  Dominic Schallert trading as schallert.com e.U.    1024   business

In [33]:
AS_pb_joined.info() #identify the datatype of the merged dataset

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10705 entries, 0 to 10704
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   prb_id   10705 non-null  int32 
 1   ASN      10705 non-null  string
 2   Country  10705 non-null  object
 3   Name     10705 non-null  object
 4   NumIPs   10705 non-null  int32 
 5   type     10705 non-null  object
dtypes: int32(2), object(3), string(1)
memory usage: 501.8+ KB


In [34]:
print(AS_pb_joined[AS_pb_joined.prb_id == AS_pb_joined.prb_id.max()]) #identify max value of the prb_id

       prb_id      ASN Country               Name   NumIPs      type
6452  1000246  AS14061      US  DigitalOcean, LLC  2163200  business


In [35]:
#add new column for determining EU or non-EU
# read text file
my_file = open("EU_country.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
EU_country_list = data.split("\n")
print(EU_country_list)
my_file.close()

['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE']


In [36]:
#filter hosting type of AS_pb_joined
AS_pb_joined = AS_pb_joined[AS_pb_joined['type'] == 'hosting']

In [37]:
#filter the AS_pb_joined only to EU country
EU_AS_pb_joined = AS_pb_joined[AS_pb_joined.Country.isin(EU_country_list)]

In [38]:
#show 10 first dataset of EU_AS_pb_joined
EU_AS_pb_joined.head(10)

prb_id      ASN Country                       Name  NumIPs     type
1637      57  AS20621      DE            PlusServer GmbH   10240  hosting
2451      92  AS12859      NL                     BIT BV   66816  hosting
2452    6234  AS12859      NL                     BIT BV   66816  hosting
2453    6237  AS12859      NL                     BIT BV   66816  hosting
2454   32445  AS12859      NL                     BIT BV   66816  hosting
2522     107  AS25596      NL  Cambrium IT Services B.V.   18432  hosting
2523     989  AS25596      NL  Cambrium IT Services B.V.   18432  hosting
2524    4052  AS25596      NL  Cambrium IT Services B.V.   18432  hosting
2525    4273  AS25596      NL  Cambrium IT Services B.V.   18432  hosting
2526    4680  AS25596      NL  Cambrium IT Services B.V.   18432  hosting

In [39]:
#describe the dataset
EU_AS_pb_joined.describe

<bound method NDFrame.describe of         prb_id       ASN Country                     Name  NumIPs     type
1637        57   AS20621      DE          PlusServer GmbH   10240  hosting
2451        92   AS12859      NL                   BIT BV   66816  hosting
2452      6234   AS12859      NL                   BIT BV   66816  hosting
2453      6237   AS12859      NL                   BIT BV   66816  hosting
2454     32445   AS12859      NL                   BIT BV   66816  hosting
...        ...       ...     ...                      ...     ...      ...
10603    54951  AS205766      DE             Jonas Pasche    1792  hosting
10656    55773  AS201525      BG           HZ Hosting Ltd    1536  hosting
10663  1000024   AS51167      DE             Contabo GmbH  177152  hosting
10664  1000183   AS51167      DE             Contabo GmbH  177152  hosting
10678  1000116   AS44220      RO  Parfumuri Femei.com SRL    3328  hosting

[339 rows x 6 columns]>

In [40]:
EU_AS_pb_joined[EU_AS_pb_joined['NumIPs'] == '0'].count()  #identify row with NumIPs = 0

prb_id     0
ASN        0
Country    0
Name       0
NumIPs     0
type       0
dtype: int64

There are 339 rows which filtered by:
Type = Hosting;
Country = in Europe;
And within the dataset, the NumIPs = 0 is already excluded.

In [41]:
#sort the ASN's in ascending order 
sorted_df = EU_AS_pb_joined.sort_values('ASN', key=lambda x: x.str.extract('(\d+)').squeeze().astype(int))
print(sorted_df)

       prb_id       ASN Country                              Name  NumIPs  \
5116     6131    AS5404      AT        conova communications GmbH    8448   
5115     2732    AS5404      AT        conova communications GmbH    8448   
8153    32726    AS5430      DE  freenet Datenkommunikations GmbH  722432   
8152    15517    AS5430      DE  freenet Datenkommunikations GmbH  722432   
10328   51157    AS5521      DE                   PlusServer GmbH   22528   
...       ...       ...     ...                               ...     ...   
2911    22203  AS203953      DK                         Hiper A/S   17408   
2915    51413  AS203953      DK                         Hiper A/S   17408   
2910      298  AS203953      DK                         Hiper A/S   17408   
2912    23321  AS203953      DK                         Hiper A/S   17408   
10603   54951  AS205766      DE                      Jonas Pasche    1792   

          type  
5116   hosting  
5115   hosting  
8153   hosting  
8152   

In [42]:
#show 3 first data of the sorted dataset
sorted_df[['ASN', 'Name', 'Country']].head(3)

ASN                              Name Country
5116  AS5404        conova communications GmbH      AT
5115  AS5404        conova communications GmbH      AT
8153  AS5430  freenet Datenkommunikations GmbH      DE

In [43]:
#show 3 last data of the sorted dataset
sorted_df[['ASN', 'Name', 'Country']].tail(3)

ASN          Name Country
2910   AS203953     Hiper A/S      DK
2912   AS203953     Hiper A/S      DK
10603  AS205766  Jonas Pasche      DE

In [44]:
unique_sorted_df = sorted_df.ASN.unique()
len(unique_sorted_df)

145

There are 145 unique ASN from 339 rows of sorted dataset.

### Task C
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way

In [45]:
#store the timestamp to integer (or convert as you want as required)

In [46]:
#decoded_line = filter(lambda decoded_line: decoded_line['mver'], decoded_line)

In [47]:
ripe_df = pd.DataFrame(decoded_line)
ripe_df.tail(100)

fw  lts      dst_name  af      dst_addr        src_addr proto  ttl  size  \
0  4790   41  199.9.14.201   4  199.9.14.201  192.168.21.101  ICMP   56    20   
1  4790   41  199.9.14.201   4  199.9.14.201  192.168.21.101  ICMP   56    20   
2  4790   41  199.9.14.201   4  199.9.14.201  192.168.21.101  ICMP   56    20   

               result  dup  rcvd  sent        min        max        avg  \
0  {'rtt': 41.258504}    0     3     3  41.125641  41.258504  41.182985   
1  {'rtt': 41.164809}    0     3     3  41.125641  41.258504  41.182985   
2  {'rtt': 41.125641}    0     3     3  41.125641  41.258504  41.182985   

   msm_id  prb_id   timestamp msm_name            from  type  step  
0    1010    4861  1646093211     Ping  188.163.120.63  ping   240  
1    1010    4861  1646093211     Ping  188.163.120.63  ping   240  
2    1010    4861  1646093211     Ping  188.163.120.63  ping   240

The data response is not consistent, there are lists with 'mver' column and there are also lists without 'mver' column which caused the mapping in aboved code is not optimal. A manual mapping column from json responses to panda dataframe should be done to avoid 'mver' reading and to select several data needed only for the assignment.

In [48]:
bz2Filename = 'ping-2022-03-01T0000.bz2'
bz2File     = bz2.open(bz2Filename, 'rt') 

# List of tuples
# https://stackoverflow.com/questions/28056171/how-to-build-and-fill-pandas-dataframe-from-for-loop
tuple_list = []
probes_in_RIPE_list = []
#read first 10 lines to estimate total loading time
totalNrOfLines = 0
filteredNrOfLines = 0

start  = time.time()
for line in bz2File:
    #print(line)
    decoded_line = json.loads(line)
    if "af" in decoded_line and "dst_addr" in decoded_line and "prb_id" in decoded_line and "avg" in decoded_line: 
        if(decoded_line["af"] == 4 and (decoded_line["prb_id"] in sorted_df["prb_id"]) and (decoded_line["avg"] > 0)):
            filteredNrOfLines += 1
            probes_in_RIPE_list.append(decoded_line["prb_id"])
            tuple_list.append((decoded_line["dst_addr"],decoded_line["prb_id"],decoded_line["avg"]))
    #print(decoded_line)
    totalNrOfLines += 1
    #if totalNrOfLines > 100000:
    #    break

duration = round(time.time() - start,2)
print("\nbz2 file:" )
print("Loading " + str(totalNrOfLines) +  " lines took: "  + str(duration) + " seconds")
print("Selected " + str(filteredNrOfLines) +  " lines")



bz2 file:
Loading 28662238 lines took: 681.24 seconds
Selected 648635 lines


In [49]:
#map column from json response to panda dataframe
#df_final = pd.json_normalize(decoded_line)
#df_final.head()

In [50]:
probes_set = set(probes_in_RIPE_list) #set probes in RIPE list
len(probes_set)

82

In [51]:
filtered_RIPE_df = pd.DataFrame(tuple_list, columns=('dst_addr', 'prb_id', 'avg')) #mapping dst_addr, prb_id and avg in a new dataframe

In [52]:
filtered_RIPE_df.shape

(648635, 3)

In [53]:
filtered_RIPE_df.head() #show filtered RIPE with dst_addr, prb_id and avg columns

dst_addr  prb_id        avg
0   193.0.19.60   10603  34.819981
1  129.13.73.54   10603  44.838081
2  81.91.95.116   10603  45.931803
3  5.188.108.64   10603  18.466986
4  80.67.181.29   10603  38.368946

In [54]:
filtered_RIPE_df.isnull().any() #check wether there are null value in the dataset

dst_addr    False
prb_id      False
avg         False
dtype: bool

In [55]:
filtered_RIPE_df['dst_addr'].describe()

count           648635
unique            1112
top       202.12.27.33
freq              1220
Name: dst_addr, dtype: object

In [56]:
filtered_RIPE_df['dst_addr'].min()

'1.0.0.1'

There are no 0.0.0.0 dst_addr which means the dataset for dst_addr is already valid

In [57]:
filtered_RIPE_df['prb_id'].describe()

count    648635.000000
mean       6644.253739
std         554.052628
min        2522.000000
25%        6471.000000
50%        6747.000000
75%        6879.000000
max       10678.000000
Name: prb_id, dtype: float64

Minimum and maximum value of prb_id is 2522 and 10678 respectively which means the dataset is valid

In [58]:
filtered_RIPE_df['avg'].describe()

count    648635.000000
mean        124.421385
std          94.166873
min           0.025559
25%          38.951239
50%         103.415823
75%         197.647896
max        1138.807999
Name: avg, dtype: float64

Minimum and maximum value of avg is 0.025559 and 1138.807999 which means the dataset is valid

In [59]:
#combine filtered_RIPE_df with sorted_df
#merge the AS dataset and Probe data
filtered_RIPE_df = filtered_RIPE_df.merge(sorted_df, left_on='prb_id', right_on='prb_id', how='inner')
filtered_RIPE_df = filtered_RIPE_df.replace(np.nan,'', regex=True) 

In [60]:
filtered_RIPE_df.head()

dst_addr  prb_id         avg      ASN Country             Name  \
0   217.28.166.213    6584   17.631136  AS35277      RO  LLHOST INC. SRL   
1     96.126.70.90    6584  224.582221  AS35277      RO  LLHOST INC. SRL   
2  109.168.125.243    6584   15.618416  AS35277      RO  LLHOST INC. SRL   
3    217.25.179.62    6584   29.279891  AS35277      RO  LLHOST INC. SRL   
4    217.31.202.20    6584   26.894860  AS35277      RO  LLHOST INC. SRL   

   NumIPs     type  
0    1536  hosting  
1    1536  hosting  
2    1536  hosting  
3    1536  hosting  
4    1536  hosting

In [61]:
#convert ipaddress from IPv4 format to integer format
import ipaddress
filtered_RIPE_df["dst_ipint"] = filtered_RIPE_df["dst_addr"].apply(lambda x:int(ipaddress.IPv4Address(x)))

In [62]:
filtered_RIPE_df.head() #show the new filtered_RIPE_df.head and now there is new column of IP with integer format

dst_addr  prb_id         avg      ASN Country             Name  \
0   217.28.166.213    6584   17.631136  AS35277      RO  LLHOST INC. SRL   
1     96.126.70.90    6584  224.582221  AS35277      RO  LLHOST INC. SRL   
2  109.168.125.243    6584   15.618416  AS35277      RO  LLHOST INC. SRL   
3    217.25.179.62    6584   29.279891  AS35277      RO  LLHOST INC. SRL   
4    217.31.202.20    6584   26.894860  AS35277      RO  LLHOST INC. SRL   

   NumIPs     type   dst_ipint  
0    1536  hosting  3642533589  
1    1536  hosting  1618888282  
2    1536  hosting  1839758835  
3    1536  hosting  3642340158  
4    1536  hosting  3642739220

In [63]:
filtered_RIPE_df.count() #filtered_RIPE_df for EU country only since it is merged with AS dataset that already filtered by EU only

dst_addr     22839
prb_id       22839
avg          22839
ASN          22839
Country      22839
Name         22839
NumIPs       22839
type         22839
dst_ipint    22839
dtype: int64

From total 648635 data in the 1 hour dataset, the data which already filtered by hosting type and used probe id (prb_id) is 22839 which is only 3.5% of total data in 1 hour.

Now, filter the data to identify data which target IPv4 is from an EU country only with reference to ipv4_df dataset.

In [82]:
#first filter ipv4_df for EU only
EU_ipv4_df = ipv4_df[ipv4_df.country_code.isin(EU_country_list)]
EU_ipv4_df.head()

ip_from     ip_to country_code country_name
88   28499968  28508159           ES        Spain
96   28536832  28540927           FR       France
105  33554432  34603263           FR       France
106  34603264  34603519           PL       Poland
107  34603520  34603775           CZ      Czechia

In [83]:
EU_ipv4_df.count() #there are 65904 rows for ipv4 in EU only

ip_from         65904
ip_to           65904
country_code    65904
country_name    65904
dtype: int64

In [84]:
filtered_RIPE_df['dst_ipint'].info()

<class 'pandas.core.series.Series'>
Int64Index: 22839 entries, 0 to 22838
Series name: dst_ipint
Non-Null Count  Dtype
--------------  -----
22839 non-null  int64
dtypes: int64(1)
memory usage: 356.9 KB


In [85]:
len(EU_ipv4_df)

65904

In [119]:
dict(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']))

{28499968: 28508159,
 28536832: 28540927,
 33554432: 34603263,
 34603264: 34603519,
 34603520: 34603775,
 34603776: 34604031,
 34604544: 34605055,
 34605056: 34605311,
 34605312: 34605567,
 34605568: 34605823,
 34605824: 34606079,
 34607104: 34607359,
 34607360: 34607615,
 34607616: 34607871,
 34607872: 34608127,
 34608640: 34608895,
 34608896: 34609151,
 34609152: 34609407,
 34609408: 34609663,
 34609920: 34610175,
 34610432: 34611711,
 34611712: 34611967,
 34612736: 34613247,
 34613504: 34613759,
 34613760: 34614271,
 34614272: 34614527,
 34614784: 34615295,
 34615552: 34615807,
 34615808: 34616063,
 34616320: 34616575,
 34616576: 34616831,
 34616832: 34617087,
 34617088: 34617343,
 34618368: 34618623,
 34618624: 34619135,
 34619136: 34619391,
 34619392: 34619647,
 34619648: 34619903,
 34619904: 34620927,
 34620928: 34621439,
 34622976: 34623487,
 34623488: 34623999,
 34624512: 34625023,
 34625024: 34625535,
 34625792: 34626047,
 34626048: 34626559,
 34626816: 34627071,
 34627072: 34

In [122]:
numbers = filtered_RIPE_df['dst_ipint']
ranges = dict(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to'])) 
score = {} # dict to store results in

for number in numbers:
    for range_name in sorted(ranges):
        range_list = ranges[range_name]
        groups = zip(*[iter(range_list)] * 2)
        if any(start <= number < end for start,end in groups):
            df3 = pd.DataFrame(numbers)
            #scores.setdefault(range_name, 0) += 1

TypeError: 'int' object is not iterable

In [118]:
#x = 0
#for v in filtered_RIPE_df['dst_ipint']:
#    if x < len(EU_ipv4_df):
#        if v in range (EU_ipv4_df['ip_from'][x], EU_ipv4_df['ip_to'][x]):
#            df3 = True
#        else:
#            x = x + 1
#        if v in range(EU_ipv4_df['ip_from'][x], EU_ipv4_df['ip_to'][x]):
#         return True
#        else:
#         return False
#        x = x + 1
    #EU_ipv4_df = EU_ipv4_df[(EU_ipv4_df['ip_from'] < v) &  (EU_ipv4_df['ip_to'] > v)]

KeyError: 0

In [102]:
EU_ipv4_df.count()

ip_from         65904
ip_to           65904
country_code    65904
country_name    65904
dtype: int64

TypeError: 'ipv4_in_EU' is an invalid keyword argument for print()

In [78]:
i = filtered_RIPE_df['dst_ipint']
a = EU_ipv4_df['ip_from']
b = EU_ipv4_df['ip_to']

def check(i, a, b): # check if i is in the range. # return True or False
    if i in range(a, b):
        return True
    #else:
     #   return False

In [70]:
ip_range = list(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']))

In [71]:
filtered_RIPE_df[filtered_RIPE_df['dst_ipint'].isin(range(ip_range))]

TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
EU_ipv4_df['ip_range'] = list(zip(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']))

In [ ]:
EU_ipv4_df['ip_range'] = EU_ipv4_df.apply(range(EU_ipv4_df['ip_from'].astype(int), EU_ipv4_df['ip_to'].astype(int)))

In [ ]:
def check(filtered_RIPE_df['dst_ipint'], EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']): # check if i is in the range. # return True or False
    if filtered_RIPE_df['dst_ipint'] in range(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to']):
        return True
    else:
        return False

In [ ]:
#dict(zip(df.state, df.name))
EU_ipv4_df['ip_range'] = dict(zip(EU_ipv4_df.ip_from, EU_ipv4_df.ip_to))

In [ ]:
a = filtered_RIPE_df['dst_addr'].reference.values
bh = EU_ipv4_df['ip_to'].end.values
bl = EU_ipv4_df['ip_from'].start.values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

EU_df = pd.DataFrame(
    np.column_stack([filtered_RIPE_df['dst_addr'][i], EU_ipv4_df['ip_to'].values[j]]),
    columns=filtered_RIPE_df['dst_addr'].columns.append(EU_ipv4_df['ip_to'].columns)
)

# if you need not elements not in range

#filtered_RIPE_df.merge(EU_df, on='reference', how='left')

#filtered_RIPE_df['is_from_EU'] = filtered_RIPE_df['dst_ipint'].between(EU_ipv4_df['ip_from'], EU_ipv4_df['ip_to'])  
#filtered_RIPE_df['is_from_EU'] = filtered_RIPE_df['dst_ipint'] >= EU_ipv4_df['ip_from'] #& (filtered_RIPE_df['dst_ipint'] < EU_ipv4_df['ip_to'])

country n x probe id matrix -> average latency

d. Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one ncountries ×m matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)

1. Combine all data from each hour (full day data)
2. Calcualte the average latency for each country - AS combination 
3. Store results into one n countries x m matrix (m = probe id) or ASN?
4. Find the minimum average latency for each country
(do visualization for the datasets)

e. Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.

1. Find the total latency for all countries * (?)
2. Find the average latency for each country
3. Sort/folter based on four datacenters (Question: datacenters is attached to the country or to the probe id?)
4. Average latency for each country.

5. Report findings
6. Report procedure to obtain them